<a href="https://colab.research.google.com/github/OriolOriolOriol/Perseus/blob/main/Perseus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint

Importato le librerie necessari per creare il modello.
Ecco di seguito cosa fanno:
